In [1]:
#import dependancies
import pandas as pd
import numpy as np
from sklearn import svm, cross_validation as cv
from sklearn.ensemble import RandomForestClassifier as RFC
from numpy.lib.stride_tricks import as_strided as ast
from skimage.feature import hog

In [2]:
#get_features function using hog method
def get_features(imgs):
    feats = np.empty([imgs.shape[0], 512])
    k = 0
    
    for img in imgs:
        img_feats = hog(img, orientations=8, pixels_per_cell=(5,5), cells_per_block=(2,2))
        
        feats[k,:] = img_feats
        k = k + 1
        
    return feats


In [3]:
# The competition datafiles are in the directory ../input
# Read competition data files:
train = pd.read_csv("C:/Users/Siddhant/Desktop/notebooks/digit_rec/input/train.csv")
test  = pd.read_csv("C:/Users/Siddhant/Desktop/notebooks/digit_rec/input/test.csv")

labels = train[[0]].values.ravel()
train = train.iloc[:,1:].values


In [4]:
# Rescale data
train = train.astype(np.float32)
train = np.multiply(train, 1/255.0)

test = test.values.astype(np.float32)
test = np.multiply(test, 1/255.0)

In [5]:
# Feature test
train = np.reshape(train, [train.shape[0], 28, 28])
test = np.reshape(test, [test.shape[0], 28, 28])
train_feats = get_features(train)
test_feats = get_features(test)

In [6]:
classifier = RFC(n_estimators=200, max_depth=None, n_jobs=-1)

In [7]:
# Crossvalidation test
scores = cv.cross_val_score(classifier, train_feats, labels, cv=5)
print("Accuracy = %0.2f +- %0.2f" % (scores.mean(), 2 * scores.std()))

Accuracy = 0.97 +- 0.00


In [8]:
# Real thing
classifier.fit(train_feats, labels)
predicted = classifier.predict(test_feats)
np.savetxt('digit_rec_rfchog_output.csv', np.c_[range(1, 1+len(test_feats)), predicted], 
    delimiter=',', comments='', header='ImageId,Label', fmt='%d')
